In [1]:
import pandas as pd
from dataset_manager import DatasetManager
dataset_path = "./datasets"
manager = DatasetManager(dataset_path)
manager.show_datasets()

,compression,description,format,identifier,local_source,source
0,NaN,Quantidade de projetos para a construção de cr...,json,aprovacao_construcao_creche,./data/aprovacao_construcao_creche.json,http://api.pgi.gov.br/api/1/serie/2555.json
1,NaN,Apresentam informações relativas a cada ato de...,csv,riotransparente_despesas_ato_2017,./data/Open_Data_Desp_Ato_2017.csv,http://riotransparente.rio/riotransparente/arq...
2,NaN,Apresentam informações consolidadas da execuçã...,csv,riotransparente_despesas_2017,./data/Open_Data_Desp_2017.csv,http://riotransparente.rio/riotransparente/arq...
3,NaN,Educação Infantil e Ensino fundamental: número...,xls,data_rio_2004_2008,./data/data_rio_2004_2008.xls,./data/data_rio_2004_2008.xls
4,NaN,Apresentam informações relativas a cada liquid...,csv,riotransparente_favorecidos_2016,./data/Open_Data_Favorecidos_2016.csv,http://riotransparente.rio/riotransparente/arq...
5,NaN,Apresentam informações consolidadas da execuçã...,csv,riotransparente_despesas_2016,./data/Open_Data_Desp_2016.csv,http://riotransparente.rio/riotransparente/arq...
6,NaN,Quantidade de creches e pré-escolas cujas obra...,json,construcao_creche,./data/construcao_creche.json,http://api.pgi.gov.br/api/1/serie/2620.json
7,NaN,Apresentam informações consolidadas da execuçã...,csv,riotransparente_despesas_2018,./data/Open_Data_Desp_2018.csv,http://riotransparente.rio/riotransparente/arq...
8,NaN,Apresentam informações relativas aosdados cada...,csv,riotransparente_contratos_2015,./data/Open_Data_Contratos_2015.csv,http://riotransparente.rio/riotransparente/arq...
9,NaN,Apresentam informações relativas aosdados cada...,csv,riotransparente_contratos_2018,./data/Open_Data_Contratos_2018.csv,http://riotransparente.rio/riotransparente/arq...


In [2]:
manager.prepare_dataset()

05-18 11:11 - DatasetManager - prepare_dataset - INFO - Preparing aprovacao_construcao_creche ...
05-18 11:11 - DatasetManager - prepare_dataset - INFO - aprovacao_construcao_creche is ready to use!
05-18 11:11 - DatasetManager - prepare_dataset - INFO - Preparing riotransparente_despesas_ato_2017 ...
05-18 11:11 - DatasetManager - prepare_dataset - INFO - riotransparente_despesas_ato_2017 is ready to use!
05-18 11:11 - DatasetManager - prepare_dataset - INFO - Preparing riotransparente_despesas_2017 ...
05-18 11:11 - DatasetManager - prepare_dataset - INFO - riotransparente_despesas_2017 is ready to use!
05-18 11:11 - DatasetManager - prepare_dataset - INFO - Preparing data_rio_2004_2008 ...
05-18 11:11 - DatasetManager - prepare_dataset - INFO - data_rio_2004_2008 is ready to use!
05-18 11:11 - DatasetManager - prepare_dataset - INFO - Preparing riotransparente_favorecidos_2016 ...
05-18 11:11 - DatasetManager - prepare_dataset - INFO - riotransparente_favorecidos_2016 is ready to us

In [3]:
dataset = manager.get_dataset("riotransparente_contratos_2017")

In [11]:
dataset.load_as_pandas(error_bad_lines=False, encoding='latin-1', sep=";")

,Ano instrumento,Nr instrumento,Situação,Espécie,Código favorecido,Favorecido,Processo instrutivo,Data início previsto,Data fim previsto,Programa de trabalho,...,Tipo Administração,Dir Ind,Exercício origem,Instrumento Origem,Número do processo de origem,Órgão executor de origem,Descrição do órgão executor de origem,Novo codigo do favorecido,Novo favorecido,Data da mudança
0,2017,15032,ENCERRADO,Termo de Execução,7545455000131,THALI COMERCIO E SERVICOS DE MATERIAL DE LIMPE...,1.004334e+10,01/01/2017,31/12/2018,15.09.13.391.0387.2124,...,SECRETARIA,DIRETA,2015.0,10032.0,1.004334e+10,4100.0,GABINETE DO PREFEITO,NaN,NaN,NaN
1,2017,16,ATIVO,Termo de Execução,4641617000147,INSTITUTO BRASILEIRO DE ESTUDOS ESPECIALIZADOS...,1.500055e+11,02/01/2017,15/02/2017,16.17.27.812.0032.2068,...,SECRETARIA,DIRETA,2016.0,16.0,1.500055e+11,2500.0,SECRETARIA MUNICIPAL DE ESPORTES E LAZER,NaN,NaN,NaN
2,2017,35,ATIVO,Termo de Execução,3139962000114,UNIAO ESPORTIVA VILA OLIMPICA DA MARE UEVOM,1.500068e+11,02/01/2017,28/12/2019,16.17.27.812.0032.2068,...,SECRETARIA,DIRETA,2015.0,17.0,1.500068e+11,2500.0,SECRETARIA MUNICIPAL DE ESPORTES E LAZER,NaN,NaN,NaN
3,2017,45,ENCERRADO,Convênio sem Valor,33663683002674,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,7.000898e+10,28/04/2017,27/04/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,51,ENCERRADO,Contrato,9369104000142,AVALIA QUALIDADE EDUCACIONAL LTDA,7.004612e+10,19/05/2017,30/06/2018,16.01.12.361.0316.1301,...,SECRETARIA,DIRETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017,81,ATIVO,Convênio sem Valor,33555921000170,FACULDADES CATOLICAS,7.003854e+10,01/08/2017,31/07/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2017,94,ATIVO,Convênio sem Valor,21178963000150,INSTITUTO CCR,7.002538e+10,04/08/2017,03/08/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,173,ENCERRADO,Convênio,33591793000209,ASSOCIACAO FRANCISCANA NOSSA SENHORA DO AMPARO,7.004618e+10,19/09/2017,30/06/2018,16.01.12.306.9000.5030,...,SECRETARIA,DIRETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017,188,ATIVO,Convênio sem Valor,42401588000135,SOCIEDADE DE EDUCAÌO E ASSISTENCIA REALENGO,7.005322e+10,02/10/2017,31/07/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017,220,ENCERRADO,Convênio,7501534000140,CRECHE E ESCOLA COMUNITARIA GENTE INOCENTE,7.005775e+10,23/10/2017,30/06/2018,16.01.12.306.9000.5030,...,SECRETARIA,DIRETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Mesmo que
#import pandas as pd
#pd.read_csv("data/Open_Data_Contratos_2017.csv", error_bad_lines=False, encoding='latin-1', sep=";")

,Ano instrumento,Nr instrumento,Situação,Espécie,Código favorecido,Favorecido,Processo instrutivo,Data início previsto,Data fim previsto,Programa de trabalho,...,Tipo Administração,Dir Ind,Exercício origem,Instrumento Origem,Número do processo de origem,Órgão executor de origem,Descrição do órgão executor de origem,Novo codigo do favorecido,Novo favorecido,Data da mudança
0,2017,15032,ENCERRADO,Termo de Execução,7545455000131,THALI COMERCIO E SERVICOS DE MATERIAL DE LIMPE...,1.004334e+10,01/01/2017,31/12/2018,15.09.13.391.0387.2124,...,SECRETARIA,DIRETA,2015.0,10032.0,1.004334e+10,4100.0,GABINETE DO PREFEITO,NaN,NaN,NaN
1,2017,16,ATIVO,Termo de Execução,4641617000147,INSTITUTO BRASILEIRO DE ESTUDOS ESPECIALIZADOS...,1.500055e+11,02/01/2017,15/02/2017,16.17.27.812.0032.2068,...,SECRETARIA,DIRETA,2016.0,16.0,1.500055e+11,2500.0,SECRETARIA MUNICIPAL DE ESPORTES E LAZER,NaN,NaN,NaN
2,2017,35,ATIVO,Termo de Execução,3139962000114,UNIAO ESPORTIVA VILA OLIMPICA DA MARE UEVOM,1.500068e+11,02/01/2017,28/12/2019,16.17.27.812.0032.2068,...,SECRETARIA,DIRETA,2015.0,17.0,1.500068e+11,2500.0,SECRETARIA MUNICIPAL DE ESPORTES E LAZER,NaN,NaN,NaN
3,2017,45,ENCERRADO,Convênio sem Valor,33663683002674,UNIVERSIDADE FEDERAL DO RIO DE JANEIRO,7.000898e+10,28/04/2017,27/04/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,51,ENCERRADO,Contrato,9369104000142,AVALIA QUALIDADE EDUCACIONAL LTDA,7.004612e+10,19/05/2017,30/06/2018,16.01.12.361.0316.1301,...,SECRETARIA,DIRETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017,81,ATIVO,Convênio sem Valor,33555921000170,FACULDADES CATOLICAS,7.003854e+10,01/08/2017,31/07/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2017,94,ATIVO,Convênio sem Valor,21178963000150,INSTITUTO CCR,7.002538e+10,04/08/2017,03/08/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,173,ENCERRADO,Convênio,33591793000209,ASSOCIACAO FRANCISCANA NOSSA SENHORA DO AMPARO,7.004618e+10,19/09/2017,30/06/2018,16.01.12.306.9000.5030,...,SECRETARIA,DIRETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017,188,ATIVO,Convênio sem Valor,42401588000135,SOCIEDADE DE EDUCAÌO E ASSISTENCIA REALENGO,7.005322e+10,02/10/2017,31/07/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017,220,ENCERRADO,Convênio,7501534000140,CRECHE E ESCOLA COMUNITARIA GENTE INOCENTE,7.005775e+10,23/10/2017,30/06/2018,16.01.12.306.9000.5030,...,SECRETARIA,DIRETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
